## load module

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import random
import json
import warnings
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.manifold import TSNE

############### sklearn ###############
from sklearn.datasets import load_boston, load_diabetes
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, cross_validate, KFold, cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline

import xgboost as xgb
from tqdm import tqdm
from nltk.cluster import KMeansClusterer
import nltk
import csv
from datetime import datetime

# pytorch
import torch
from torch import nn
from datasets import Dataset
from transformers import AutoConfig, AutoModel, AutoTokenizer
from torch.utils.tensorboard import SummaryWriter
############### pytorch ###############
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, IterableDataset, TensorDataset
from torch.utils.tensorboard import SummaryWriter
############### sklearn ###############

from transformers import BertTokenizer, BertModel, AutoAdapterModel, AdapterTrainer
from transformers.modeling_outputs import TokenClassifierOutput
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers.adapters import BertAdapterModel, AutoAdapterModel, AdapterArguments
from tokenizers.normalizers import BertNormalizer
from transformers.trainer_utils import get_last_checkpoint
from transformers.modeling_outputs import TokenClassifierOutput

from autogluon.tabular import TabularDataset, TabularPredictor

warnings.filterwarnings('ignore')
np.random.seed(123)
from datasets import load_dataset, Dataset

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModel,
    DataCollatorForLanguageModeling,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    set_seed,
    AdamW
)

pd.set_option('mode.chained_assignment', None)
pd.set_option('max_colwidth', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

NOTE: Redirects are currently not supported in Windows or MacOs.


## utils

In [2]:
def gen_text_seq(df):
    text_li=[]
    for i in range(df.shape[0]):
        li = list(df.iloc[i, 3:8])
        # print(li)
        text = f"solution treatment temperature/°C |{li[0]}| solution treatment time/min |{li[1]}| cold rolling/% |{li[2]}| annealing temperature/°C |{li[3]}| annealing time/min|{li[4]}"
        text_li.append(text)   
    return text_li

In [5]:
def load_old_data(pred_prop='all', split_ratio=0.75, seed=666):
    """
    Load experiment dataset with different features combinations.
    ------------------------------    
    prop
        ['Tensile_value', 'Yield_value', 'Elongation_value', 'all']
    fes
        ['com', 'actions', 'text', 'embeddings']
    """
    df = pd.read_excel('./dataset/experiment_data/old_text_data.xlsx')
    df.fillna('', inplace=True)
    df['Text'] = df['Text'] + ' ' + df['Text_addition']
    
    # property selection
    prop_all = ['Tensile_value', 'Yield_value', 'Elongation_value']
    if pred_prop == 'all':
        drop_col = []
    else:
        drop_col = list(set(prop_all) - set([pred_prop]))
        
    data = df.drop(columns=drop_col+['Text_addition']).copy()
    data.reset_index(drop=True, inplace=True)

    for _ in range(50):
        data = data.sample(frac=1.0, random_state=seed)
    
    # df_train, df_test = np.split(data.sample(frac=1, random_state=seed, ignore_index=True), [int(split_ratio*len(data))])
    # df_test.reset_index(drop=True, inplace=True)
    
    return data

In [6]:
df = load_old_data(pred_prop='Tensile_value', split_ratio=0.75, seed=666)
df.head()

,Tensile_value,Text
142,784.0,Six experimental interstitially-alloyed Cr-Mn austenitic stainless steels with chemical compositions given in Table 1 were ingot-cast in a laboratory vacuum induction furnace under a protective n...
861,989.0,"The high-nitrogen austenitic stainless steel studied in the present investigation was supplied by M/s Jindal Steel Ltd. (JSL), Hisar, India, as forged steel block of 30 cm × 5 cm × 3.5 cm size. Fl..."
558,955.0,"As-cast billets were homogenized at 1100 °C for 3 h and subsequently machined into cubic block specimens with a size of 70 mm × 70 mm × 70 mm. Before forging, the specimen was heated to 1100 °C an..."
191,1630.2,"To perform the thermomechanical treatment, samples were cut into 10*50*100 mm plates. Steel plates were heated at 850 °C for 60 min in a furnace with a heating rate of about 14 °C/min, before quen..."
181,1112.0,"Experimental slabs were hot rolled and directly quenched using a laboratory hot rolling mill. Based on the austenite grain growth studies described below, 55 mm thick slabs were reheated at 1225 °..."


In [7]:
def auto_reg(label, data_train, data_test, metric='r2', time_min=1):
    predictor = TabularPredictor(label=label, eval_metric=metric).fit(data_train, 
                                time_limit=time_min*60, presets='best_quality')
    # add learderboard
    print(predictor.leaderboard(data_test, extra_metrics=['r2'], silent=True))
    
    # performance on training data
    train_performance = predictor.evaluate(data_train, auxiliary_metrics=True)

    train_r2, train_rmse = round(train_performance['r2'],4), round(train_performance['root_mean_squared_error'], 4)
    y_train = data_train.loc[:, label]
    y_train_pred = predictor.predict(data_train)
    
    # performance on test data
    test_performance = predictor.evaluate(data_test, auxiliary_metrics=True)
    test_r2, test_rmse = round(test_performance['r2'], 4), round(test_performance['root_mean_squared_error'], 4)
    y_test = data_test.loc[:, label]
    y_test_pred = predictor.predict(data_test)
    
    ############## plot #########################
    fig = plt.figure(figsize=(10,5), dpi=150)
    
    # on training data
    plt.subplot(121)
    plt.scatter(y_train, y_train_pred, s=15, color='#1F4B73')
    plt.plot(np.arange(0,int(max(y_test)*1.2)), np.arange(0,int(max(y_test)*1.2)), '-', color='#A2555A')
    plt.title(f'Train: $R^2$ = {train_r2}, RMSE={train_rmse}',fontdict={'family':'Times New Roman', 'size': 10})
    plt.ylabel('prediction', fontdict={'family':'Times New Roman', 'size': 14})
    plt.xlabel('True', fontdict={'family':'Times New Roman', 'size': 14})
    plt.xlim(0,int(max(y_test)*1.2))
    plt.xlim(0,int(max(y_test)*1.2))
    plt.legend(labels=['Train data','Y=X'])
    plt.grid()

    # on test data
    plt.subplot(122)
    plt.scatter(y_test, y_test_pred,s=15 , color='#1F4B73')
    plt.plot(np.arange(0,int(max(y_test)*1.2)), np.arange(0,int(max(y_test)*1.2)), '-', color='#A2555A')
    plt.title(f'Test: $R^2$ = {test_r2}, RMSE={test_rmse}',fontdict={'family':'Times New Roman', 'size': 10})
    plt.ylabel('prediction', fontdict={'family':'Times New Roman', 'size': 14})
    plt.xlabel('True', fontdict={'family':'Times New Roman', 'size': 14})
    plt.xlim(0,int(max(y_test)*1.2))
    plt.xlim(0,int(max(y_test)*1.2))
    plt.legend(labels=['Test data','Y=X'])
    plt.grid()
    # plt.savefig(f'./mlp_fig/{name}.png')
    return train_performance, test_performance

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# model_name = 'bert-base-uncased'
# model_name = 'microsoft/deberta-v3-base'
# model_name = 'm3rg-iitd/matscibert'
model_name = './../model_saved/checkpoint-140000'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddinngs(text_list):
    encoded_input = tokenizer(
        text_list, padding='max_length', max_length = 512, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output).detach().cpu().numpy()[0].tolist()

cuda


Some weights of the model checkpoint at ./../model_saved/checkpoint-140000 were not used when initializing DebertaV2Model: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## embeddings

In [3]:
def embed_exp_data():

    bert_df = pd.read_excel('./dataset/experiment_data/zhangcheng_exp_data.xlsx')
    cols_li = ['Tensile_name', 'Material', 'Tensile_unit', 'Yield_name', 'Yield_unit'] + \
        ['Elongation_name', 'Elongation_unit', 'H', 'F', 'Na', 'Mg', 'Cl', 'Ca', 'Zn', 'As', 'Zr', 'Y'] + \
        ['Bi', 'Pb', 'Ta', 'Ce', 'La', 'Sb', 'Zr', 'Fe', 'Sn']

    bert_df.drop(columns=list(set(cols_li)&set(bert_df.columns)), inplace=True)
    bert_df.reset_index(drop=True, inplace=True)
    # print(bert_df.shape)
    # print(bert_df.columns[:])
    
    tqdm.pandas(desc='Progress bar!')
    # bert_df['process_emb'] = bert_df['Text'].progress_apply(get_embeddinngs)
    bert_df['process_emb'] = bert_df['Text'].apply(get_embeddinngs)

    temp_bert_df = pd.DataFrame(pd.Series(bert_df['process_emb'][0])).T
    for i in range(1, bert_df.shape[0]):
        new_row = pd.DataFrame(pd.Series(bert_df['process_emb'][i])).T
        temp_bert_df = pd.concat([temp_bert_df, new_row], ignore_index=True, axis=0)

    df = pd.concat([bert_df, temp_bert_df], axis=1)
    df.drop(columns=['process_emb'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

## load data

In [87]:
def load_exp_data(pred_prop='all', fes=['com', 'actions'], split_ratio=0.75, seed=666):
    """
    Load experiment dataset with different features combinations.
    ------------------------------    
    prop
        ['Tensile_value', 'Yield_value', 'Elongation_value', 'all']
    fes
        ['com', 'actions', 'text', 'embeddings']
    """
    df = embed_exp_data()
    df['Text'] = gen_text_seq(df) 
    
    # property selection
    prop_all = ['Tensile_value', 'Yield_value', 'Elongation_value']
    if pred_prop == 'all':
        drop_col = []
    else:
        drop_col = list(set(prop_all) - set([pred_prop]))
    
    # features selection
    if 'com' not in fes:
        drop_col += list(df.columns)[8:-769]
    if 'actions' not in fes:
        drop_col += list(df.columns)[3:8]
    if 'text' not in fes:
        drop_col += ['Text']
    if 'embeddings' not in fes:
        drop_col += list(df.columns)[-768:]
        
    data = df.drop(columns=drop_col).copy()
    data.reset_index(drop=True, inplace=True)

    for _ in range(50):
        data = data.sample(frac=1.0, random_state=seed)
    
    df_train, df_test = np.split(data.sample(frac=1, random_state=seed, ignore_index=True), [int(split_ratio*len(data))])
    df_test.reset_index(drop=True, inplace=True)
    
    return df_train, df_test

In [7]:
df_train, df_test = load_exp_data(pred_prop='Elongation_value', fes=['embeddings'],
                                  split_ratio=0.6, seed=666)
print(df_train.shape, df_test.shape)

df_train.head()

(39, 769) (26, 769)


,Elongation_value,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,41.73,-0.916031,-0.127834,-0.412525,-1.148250,0.397523,-1.339542,0.413916,-0.331999,-0.839937,-0.017726,-0.237142,0.552550,-0.703550,-0.196383,0.964938,0.763047,0.408478,-1.437083,0.081418,-1.040954,-0.242027,0.237640,-0.416142,-1.184805,0.471824,0.912598,-1.178648,-0.339284,0.987294,0.131147,0.404637,-0.291238,-0.534806,-0.136035,-0.561124,-0.430609,-1.161139,0.540728,0.158976,0.127382,-0.515667,-0.194996,-0.083271,0.051068,0.357276,-0.439148,0.012219,0.885475,0.116124,-0.836317,0.382456,-0.985915,-0.827866,0.248264,-0.176879,0.748900,-0.294604,0.388489,0.880481,0.315506,1.552602,-0.475215,0.805777,-0.029898,-1.077119,-0.866457,0.293282,-0.007556,0.541936,0.016930,0.378152,0.142043,0.456180,-0.084118,0.352744,-0.443961,0.123427,-0.109332,-0.707241,0.336127,0.325942,0.590749,1.251289,-1.884276,-0.172657,1.881408,-1.107557,-0.589994,0.271838,-0.179294,-2.616996,-0.034784,0.754296,-0.333799,0.478040,-0.462951,-0.434281,-1.663120,-0.357730,0.856380,-0.111549,0.935525,0.180116,0.325657,0.052699,-0.351309,-0

## predict with ag

In [10]:
metric = 'r2'
time_min = 3
split_ratio = 0.7
seed = 666

### actions

In [11]:
fes = ['actions']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    print(df_train.shape, df_test.shape)
    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

No path specified. Models will be saved in: "AutogluonModels\ag-20231007_134235\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=5, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 180s
AutoGluon will save models to "AutogluonModels\ag-20231007_134235\"
AutoGluon Version:  0.6.1
Python Version:     3.9.15
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Train Data Rows:    45
Train Data Columns: 5
Label Column: Tensile_value
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1542.73, 317.62, 993.04156, 274.51637)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Gen

(45, 6) (20, 6)


	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	-0.0694	 = Validation score   (r2)
	1.33s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 166.56s of the 166.56s of remaining time.
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	-0.0694	 = Validation score   (r2)
	1.32s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 159.44s of the 159.44s of remaining time.
	0.876	 = Validation score   (r2)
	0.27s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 159.08s of the 159.08s of remaining time.
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy

KeyboardInterrupt



### text

In [ ]:
fes = ['text']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    print(df_train.shape, df_test.shape)
    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

### com

In [ ]:
fes = ['com']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    print(df_train.shape, df_test.shape)
    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

### com + actions

In [ ]:
fes = ['com', 'actions']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    print(df_train.shape, df_test.shape)
    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

### com + text

In [ ]:
fes = ['com', 'text']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    print(df_train.shape, df_test.shape)
    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

### actions+text

In [ ]:
fes = ['actions', 'text']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    print(df_train.shape, df_test.shape)
    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

### com+actions+text

In [ ]:
fes = ['com', 'actions', 'text']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    print(df_train.shape, df_test.shape)
    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

## predict with steelberta

### embeddings

In [ ]:
fes = ['embeddings']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    print(df_train.shape, df_test.shape)
    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

### com+embeddings

In [ ]:
fes = ['com', 'embeddings']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    print(df_train.shape, df_test.shape)
    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

### com+embed_tsne

In [ ]:
fes = ['com', 'embeddings']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    # tsne 
    train_embed = pd.DataFrame(TSNE(n_components=3, learning_rate='auto',
                      init='random', perplexity=3).fit_transform(df_train.iloc[:,-768:]), 
                             columns=['tsne'+str(i) for i in range(3)])
    test_embed = pd.DataFrame(TSNE(n_components=3, learning_rate='auto',
                      init='random', perplexity=3).fit_transform(df_test.iloc[:,-768:]),
                    columns=['tsne'+str(i) for i in range(3)])

    drop_cols = list(df_train.columns[-768:])
    df_train.drop(columns=drop_cols, inplace=True)
    df_test.drop(columns=drop_cols, inplace=True)
    df_test.reset_index(drop=True, inplace=True)

    df_train = pd.concat([df_train, train_embed], axis=1)
    df_test = pd.concat([df_test, test_embed], axis=1)

    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

### com+text+embeddings

In [ ]:
fes = ['com', 'text', 'embeddings']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    print(df_train.shape, df_test.shape)
    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

### com+actions+embeddings

In [ ]:
fes = ['com', 'actions', 'embeddings']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    print(df_train.shape, df_test.shape)
    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

### Text+actions+embeddings

In [ ]:
fes = ['text', 'actions', 'embeddings']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)
    print(df_train.shape, df_test.shape)
    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

## Multiple ColumnsMultiple regression

In [ ]:
metric = 'r2'
time_min=15
seed=666

### Multilabel reg

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path

class MultilabelPredictor():
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters
        ----------
        labels : List[str]
            The ith element of this list is the column (i.e. `label`) predicted by the ith TabularPredictor stored in this object.
        path : str, default = None
            Path to directory where models and intermediate outputs should be saved.
            If unspecified, a time-stamped folder called "AutogluonModels/ag-[TIMESTAMP]" will be created in the working directory to store all models.
            Note: To call `fit()` twice and save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
            Otherwise files from first `fit()` will be overwritten by second `fit()`.
            Caution: when predicting many labels, this directory may grow large as it needs to store many TabularPredictors.
        problem_types : List[str], default = None
            The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str], default = None
            The ith element is the `eval_metric` for the ith TabularPredictor stored in this object.
        consider_labels_correlation : bool, default = True
            Whether the predictions of multiple labels should account for label correlations or predict each label independently of the others.
            If True, the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous labels appearing earlier in this list (i.e. in an auto-regressive fashion).
            Set to False if during inference you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs :
            Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path=None, problem_types=None, eval_metrics=None, consider_labels_correlation=True, **kwargs):
        if len(labels) < 2:
            raise ValueError("MultilabelPredictor is only intended for predicting MULTIPLE labels (columns), use TabularPredictor for predicting one label (column).")
        if (problem_types is not None) and (len(problem_types) != len(labels)):
            raise ValueError("If provided, `problem_types` must have same length as `labels`")
        if (eval_metrics is not None) and (len(eval_metrics) != len(labels)):
            raise ValueError("If provided, `eval_metrics` must have same length as `labels`")
        self.path = setup_outputdir(path, warn_if_exist=False)
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = eval_metrics[i]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric, path=path_i, **kwargs)

    def fit(self, train_data, tuning_data=None, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i+1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...")
            predictor.fit(train_data=train_data, tuning_data=tuning_data, **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. If label columns are present in this data, they will be ignored. See documentation for `TabularPredictor.predict()`.
            kwargs :
                Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            eval_dict[label] = predictor.evaluate(data, **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path+self.multi_predictor_file, object=self)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = os.path.expanduser(path)
        if path[-1] != os.path.sep:
            path = path + os.path.sep
        return load_pkl.load(path=path+cls.multi_predictor_file)

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for label in self.labels:
            print(f"Predicting with TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

### Training

In [ ]:
df_train, df_test = load_exp_data(pred_prop='all', fes=fes,
                                  split_ratio=split_ratio, seed=seed)

labels = ['Tensile_value','Yield_value','Elongation_value']  # which columns to predict based on the others
problem_types = ['regression','regression','regression']  # type of each prediction problem (optional)
eval_metrics = ['r2','r2','r2']  # metrics used to evaluate predictions for each label (optional)
presets = ['best_quality', 'best_quality', 'best_quality']
save_path = 'agModels-predictEducationClass'  # specifies folder to store trained models (optional)

time_limit = 3  # how many seconds to train the TabularPredictor for each label, set much larger in your applications!

In [ ]:
multi_predictor = MultilabelPredictor(labels=labels, problem_types=problem_types, eval_metrics=eval_metrics, 
                                      path=save_path)

multi_predictor.fit(df_train, time_limit=time_limit*60, presets=presets)

### Evaluation

In [ ]:
evaluations = multi_predictor.evaluate(df_test)
print(evaluations)
print("Evaluated using metrics:", multi_predictor.eval_metrics)

## pred with compostion embed

In [ ]:
metric = 'r2'
time_min = 3
split_ratio = 0.7
seed = 666
fes=['com', 'embeddings']

### combed+CLS_0

In [ ]:
def ele_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_ele_embeddinngs(text_list):
    encoded_input = tokenizer(
        text_list, padding='max_length', max_length = 512, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    # return cls_pooling(model_output).detach().cpu().numpy()[0].tolist()
    return ele_pooling(model_output).detach().cpu().numpy()

def add_ele_embed(df):
    coms = list(df.columns[-786:-768])

    x = np.matrix(get_ele_embeddinngs(coms))
    w1 = list(df.iloc[0, -786:-768])
    y = np.average(x, axis=0, weights=w1)

    for i in np.arange(1, df.shape[0]):
        
        w = list(df.iloc[i, -786:-768])
        temp_y = np.average(x, axis=0, weights=w)
        y = np.vstack((y, temp_y))

    ele_df = pd.DataFrame(y, columns=['ele'+str(i) for i in range(768)])
    
    return ele_df


#### comd_tsne+embed_CLS

In [ ]:
fes = ['com', 'embeddings']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    # df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
    #                                   split_ratio=split_ratio, seed=seed)
    # print(df_train.shape, df_test.shape)
    
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)

    drop_cols = list(df_train.columns[-786:-768])
    df_train.drop(columns=drop_cols, inplace=True)
    df_test.drop(columns=drop_cols, inplace=True)

    # normal
    train_ele = add_ele_embed(df_train)
    test_ele = add_ele_embed(df_test)
    
    # tsne 
    train_ele = pd.DataFrame(TSNE(n_components=3, learning_rate='auto',
                      init='random', perplexity=3).fit_transform(add_ele_embed(df_train)), 
                             columns=['ele'+str(i) for i in range(3)])
    test_ele = pd.DataFrame(TSNE(n_components=3, learning_rate='auto',
                      init='random', perplexity=3).fit_transform(add_ele_embed(df_test)),
                    columns=['ele'+str(i) for i in range(3)])

    df_test.reset_index(drop=True, inplace=True)
    df_train = pd.concat([df_train, train_ele], axis=1)
    df_test = pd.concat([df_test, test_ele], axis=1)

    print(df_train.shape, df_test.shape)

    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

#### combed+embed_CLS

In [ ]:
fes = ['com', 'embeddings']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    # df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
    #                                   split_ratio=split_ratio, seed=seed)
    # print(df_train.shape, df_test.shape)
    
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)

    drop_cols = list(df_train.columns[-786:-768])
    df_train.drop(columns=drop_cols, inplace=True)
    df_test.drop(columns=drop_cols, inplace=True)

    # normal
    train_ele = add_ele_embed(df_train)
    test_ele = add_ele_embed(df_test)

    df_test.reset_index(drop=True, inplace=True)
    df_train = pd.concat([df_train, train_ele], axis=1)
    df_test = pd.concat([df_test, test_ele], axis=1)

    print(df_train.shape, df_test.shape)

    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

### combed+CLS_1

In [ ]:
def ele_pooling(model_output):
    return model_output.last_hidden_state[:, 1]

def get_ele_embeddinngs(text_list):
    encoded_input = tokenizer(
        text_list, padding='max_length', max_length = 512, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    # return cls_pooling(model_output).detach().cpu().numpy()[0].tolist()
    return ele_pooling(model_output).detach().cpu().numpy()

def add_ele_embed(df):
    coms = list(df.columns[-786:-768])

    x = np.matrix(get_ele_embeddinngs(coms))
    w1 = list(df.iloc[0, -786:-768])
    y = np.average(x, axis=0, weights=w1)

    for i in np.arange(1, df.shape[0]):
        
        w = list(df.iloc[i, -786:-768])
        temp_y = np.average(x, axis=0, weights=w)
        y = np.vstack((y, temp_y))

    ele_df = pd.DataFrame(y, columns=['ele'+str(i) for i in range(768)])
    
    return ele_df


#### comd_tsne+embd_CLS1

In [ ]:
fes = ['com', 'embeddings']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    # df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
    #                                   split_ratio=split_ratio, seed=seed)
    # print(df_train.shape, df_test.shape)
    
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)

    drop_cols = list(df_train.columns[-786:-768])
    df_train.drop(columns=drop_cols, inplace=True)
    df_test.drop(columns=drop_cols, inplace=True)

    # normal
    train_ele = add_ele_embed(df_train)
    test_ele = add_ele_embed(df_test)

    # tsne 
    train_ele = pd.DataFrame(TSNE(n_components=3, learning_rate='auto',
                      init='random', perplexity=3).fit_transform(add_ele_embed(df_train)), 
                             columns=['ele'+str(i) for i in range(3)])
    test_ele = pd.DataFrame(TSNE(n_components=3, learning_rate='auto',
                      init='random', perplexity=3).fit_transform(add_ele_embed(df_test)),
                    columns=['ele'+str(i) for i in range(3)])

    df_test.reset_index(drop=True, inplace=True)
    df_train = pd.concat([df_train, train_ele], axis=1)
    df_test = pd.concat([df_test, test_ele], axis=1)

    print(df_train.shape, df_test.shape)

    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

#### combed+embed_CLS1

In [ ]:
fes = ['com', 'embeddings']
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    # df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
    #                                   split_ratio=split_ratio, seed=seed)
    # print(df_train.shape, df_test.shape)
    
    df_train, df_test = load_exp_data(pred_prop=p, fes=fes,
                                      split_ratio=split_ratio, seed=seed)

    drop_cols = list(df_train.columns[-786:-768])
    df_train.drop(columns=drop_cols, inplace=True)
    df_test.drop(columns=drop_cols, inplace=True)

    # normal
    train_ele = add_ele_embed(df_train)
    test_ele = add_ele_embed(df_test)

    df_test.reset_index(drop=True, inplace=True)
    df_train = pd.concat([df_train, train_ele], axis=1)
    df_test = pd.concat([df_test, test_ele], axis=1)

    print(df_train.shape, df_test.shape)

    train_performance, test_performance = auto_reg(p, df_train, df_test,
                                                   metric=metric, time_min=time_min)

## pred with ML

In [9]:
def eval_model(y_true, y_pred):
    """
    Model evaluation for predicted results with "r2" and "rootmean squared error"
    """
    y_true = y_true.detach().cpu().numpy() if torch.is_tensor(y_true) else y_true
    y_pred = y_pred.detach().cpu().numpy() if torch.is_tensor(y_pred) else y_pred

    r2 = round(r2_score(y_true, y_pred), 5)
    rmse = round(mean_squared_error(y_true, y_pred, squared=False),5)
    mae = round(mean_absolute_error(y_true, y_pred), 5) 
    result = {"r2":r2, "rmse":rmse, "mae":mae}
    # print(f"r2: {r2} \n root_mean_squared_error: {rmse}\n")
    return result

In [24]:
gbr_reg = GradientBoostingRegressor(learning_rate=0.01,
                                    n_estimators=100,
                                    random_state=0)

svr_reg = SVR(C=0.8, epsilon=0.2)

xgb_reg = xgb.XGBRegressor(base_score=0.4, 
                           learning_rate=0.1,
                           random_state=789,
                           verbosity=0,)

rf_reg = RandomForestRegressor(n_estimators=10,
                               max_depth=2,
                               random_state=123)

mlp_reg = MLPRegressor(hidden_layer_sizes=[2, 4, 4],
                       activation='logistic',
                       random_state=456,
                       solver='lbfgs',
                       alpha=0.001,
                       batch_size=8,
                       learning_rate_init=0.005,
                       max_iter=200)

def ml_pred_exp(reg, target='Tensile_value', random_seed=666):
    """
    Using traditionanl ML model to predict single property with different festures combinations.
    """
    # load data
    train_data, test_data = load_exp_data(pred_prop=target, fes=['actions'],
                                      split_ratio=0.7, seed=random_seed)
    
#     # normalization
#     train_data[target] = train_data[target]  / train_data[target].abs().max() 
#     test_data[target] = test_data[target]  / test_data[target].abs().max()
    
#     y_train = train_data.iloc[:,0].values 
#     y_test = test_data.iloc[:,0].values
    
    # # tsne 
    # X_train = pd.DataFrame(TSNE(n_components=3, learning_rate='auto',
    #                   init='random', perplexity=3).fit_transform(train_data.iloc[:, 1:]), 
    #                          columns=['tsne'+str(i) for i in range(3)])
    # X_test = pd.DataFrame(TSNE(n_components=3, learning_rate='auto',
    #                   init='random', perplexity=3).fit_transform(test_data.iloc[:, 1:]),
    #                 columns=['tsne'+str(i) for i in range(3)])

    
    X_train, y_train = train_data.iloc[:,1:].values, train_data.iloc[:,0].values
    X_test, y_test = test_data.iloc[:,1:].values, test_data.iloc[:,0].values
    # print(train_data.shape, test_data.shape)
    regr = reg.fit(X_train, y_train)
    # preds = regr.predict(X_test)

    train_r2 = eval_model(y_train, regr.predict(X_train))['r2']
    test_r2 = eval_model(y_test, regr.predict(X_test))['r2']

    return train_r2, test_r2

In [65]:
for p in ['Tensile_value', 'Yield_value', 'Elongation_value']:
    # for m in ['gbr_reg', 'svr_reg', 'xgb_reg', 'rf_reg', 'mlp_reg']:
    for m in ['gbr_reg', 'svr_reg', 'xgb_reg', 'rf_reg', 'mlp_reg']:
        output = ml_pred_exp(reg=eval('rf_reg'), target=p, random_seed=666)
        print(p, m, ':', output)

Tensile_value gbr_reg : (0.44548, -1.37703)
Tensile_value svr_reg : (0.18982, -0.22504)
Tensile_value xgb_reg : (0.29226, -0.85956)
Tensile_value rf_reg : (0.3493, -0.39588)
Tensile_value mlp_reg : (0.39668, -0.39254)
Yield_value gbr_reg : (0.28477, -0.35381)
Yield_value svr_reg : (0.39059, -0.38815)
Yield_value xgb_reg : (0.40422, -0.28819)
Yield_value rf_reg : (0.42089, -0.61466)
Yield_value mlp_reg : (0.38594, -0.34012)
Elongation_value gbr_reg : (0.34271, -0.37262)
Elongation_value svr_reg : (0.4035, -0.5542)
Elongation_value xgb_reg : (0.30025, -0.32516)
Elongation_value rf_reg : (0.35658, -0.57345)
Elongation_value mlp_reg : (0.33407, -0.24034)


In [25]:
for h in [[4,]]:
    # for act in ['tanh', 'logistic', 'relu']:
    for act in ['relu']:
    # for act in ['logistic', 'tanh', 'relu']:
        for s in ['adam']:
            for a in [1e-4]:
                for b in [16, 32]:
                    for lr in [1e-3]:
                        for m in [600, 1000, 5000,10_000]:
                            for r in [789, 666, 123]:
                                mlp_reg = MLPRegressor(hidden_layer_sizes=h,
                                                       activation=act,
                                                       random_state=r,
                                                       solver=s,
                                                       alpha=a,
                                                       batch_size=b,
                                                       learning_rate='adaptive',
                                                       learning_rate_init=lr,
                                                       max_iter=m)
                                # print(h, act, s, a, b, lr, m, r)
                                with open('new_exp_mlp.csv', 'a+') as csvfile:  
                                    # creating a csv writer object  
                                    csvwriter = csv.writer(csvfile)
                                    
                                    train_r2, test_r2 = ml_pred_exp(reg=mlp_reg, target='Elongation_value', random_seed=r)
                                    csvwriter.writerow([train_r2, test_r2, h, act, s, a, b, lr, m, r])
                                    # print(output['r2'], h, act, s, a, b, lr, m, r)

In [ ]:
for h in [[1], [2,4,4], [2,4,8], [2,4]]:
    for act in ['tanh', 'logistic', 'relu']:
    # for act in ['logistic', 'tanh', 'relu']:
        for s in ['lbfgs', 'sgd', 'adam']:
            for a in [1e-3, 1e-4, 1e-5]:
                for b in [4, 8, 16, 32]:
                    for lr in [5e-3, 5e-4, 5e-5]:
                        for m in [200, 400, 600]:
                            for r in [456, 123, 789]:

                                mlp_reg = MLPRegressor(hidden_layer_sizes=h,
                                                       activation=act,
                                                       random_state=r,
                                                       solver=s,
                                                       alpha=a,
                                                       batch_size=b,
                                                       learning_rate_init=lr,
                                                       max_iter=m)
                                # print(h, act, s, a, b, lr, m, r)
                                with open('new_exp_mlp.csv', 'a+') as csvfile:  
                                    # creating a csv writer object  
                                    csvwriter = csv.writer(csvfile)
                                    
                                    output = ml_pred_exp(reg=mlp_reg, target='Tensile_value', random_seed=r)
                                    csvwriter.writerow([output['r2'], h, act, s, a, b, lr, m, r])
                                    # print(output['r2'], h, act, s, a, b, lr, m, r)

In [ ]:
# find best MLP parameters
# for h in [[2,4,4], [2,4,8], [2,4]]:
# with open('exp_mlp.csv', 'a+') as csvfile:  
#     # creating a csv writer object  
#     csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    # csvwriter.writerow(fields)  
        
    # writing the data rows  
    # csvwriter.writerows(rows) 

for h in [[1], [2,4,4], [2,4,8], [2,4]]:
    for act in ['tanh', 'logistic', 'relu']:
    # for act in ['logistic', 'tanh', 'relu']:
        for s in ['lbfgs', 'sgd', 'adam']:
            for a in [1e-2, 1e-1]:
                for b in [4, 8, 16, 32]:
                    for lr in [5e-3, 5e-4, 5e-5]:
                        for m in [200, 400, 600]:
                            for r in [456, 123, 789]:

                                mlp_reg = MLPRegressor(hidden_layer_sizes=h,
                                                       activation=act,
                                                       random_state=r,
                                                       solver=s,
                                                       alpha=a,
                                                       batch_size=b,
                                                       learning_rate_init=lr,
                                                       max_iter=m)
                                # print(h, act, s, a, b, lr, m, r)
                                with open('exp_mlp.csv', 'a+') as csvfile:  
                                    # creating a csv writer object  
                                    csvwriter = csv.writer(csvfile)
                                    
                                    output = ml_pred_exp(reg=mlp_reg, target='Tensile_value', random_seed=r)
                                    csvwriter.writerow([output['r2'], h, act, s, a, b, lr, m, r])
                                    # print(output['r2'], h, act, s, a, b, lr, m, r)

## pred with number entity

### pytorch

In [8]:
def eval_model(y_true, y_pred):
    """
    Model evaluation for predicted results with "r2" and "rootmean squared error"
    """
    y_true = y_true.detach().cpu().numpy() if torch.is_tensor(y_true) else y_true
    y_pred = y_pred.detach().cpu().numpy() if torch.is_tensor(y_pred) else y_pred

    r2 = round(r2_score(y_true, y_pred), 5)
    rmse = round(mean_squared_error(y_true, y_pred, squared=False), 5)
    mae = round(mean_absolute_error(y_true, y_pred), 5)
    result = {"r2":r2, "rmse":rmse, "mae":mae}
    return result

#### pt DataClass

In [ ]:
# containging text and ele features for pytorch
class CustomDataset(Dataset):
    def __init__(self, notes, targets, tokenizer):
        self.texts = notes
        # self.eles = eles.values if isinstance(eles, pd.DataFrame) else eles
        self.targets = targets
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        text = str(self.texts[idx])
        # ele = self.eles[idx]
        target = self.targets[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=64,
            return_token_type_ids=True,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

        return {
            "labels": torch.tensor(target, dtype=torch.float32).to(device),
            # "eles": torch.tensor(ele, dtype=torch.float32).squeeze(0).to(device),
            "input_ids": torch.tensor(encoding['input_ids']).squeeze(0).to(device),
            "attention_mask": torch.tensor(encoding['attention_mask']).squeeze(0).to(device),
            "token_type_ids": torch.tensor(encoding['token_type_ids']).squeeze(0).to(device),
            # 'text':text,
        }

In [ ]:
###################### CustomDataset class test
# train_dataloader = DataLoader(CustomDataset(train_data['Text'], train_data.iloc[:, 0],
#                                             tokenizer=tokenizer), batch_size=2)
train_data, test_data = load_exp_data(pred_prop='Elongation_value', fes=['text'],
                                      split_ratio=0.7, seed=666)
print(train_data.shape, test_data.shape)

val_dataloader = DataLoader(CustomDataset(test_data['Text'], test_data.iloc[:, 0],
                                            tokenizer=tokenizer), batch_size=2)
train_dataloader = DataLoader(CustomDataset(train_data['Text'], train_data.iloc[:, 0],
                                            tokenizer=tokenizer), batch_size=2)

for batch, s in enumerate(val_dataloader):
    print(s['input_ids'].shape)
    print(s['attention_mask'].shape)
    if batch == 0:
        break

#### pt model

In [32]:
class CustomModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.name = model
        self.transformer= AutoModel.from_pretrained(self.name)
        self.text_linear = nn.Sequential(
            # nn.Linear(768, 3),        
            # nn.Linear(3, 1),
            # nn.Sigmoid(),
            nn.MultiheadAttention(768, 1, dropout=0.0),
        )
        self.gamma = nn.Parameter(torch.ones(128, 1))        
        self.attention_layer = nn.MultiheadAttention(768, 1, dropout=0.1)
        self.encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=768, nhead=1), 1,
                                             norm=None, enable_nested_tensor=True)
        
#         encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
# transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
# src = torch.rand(10, 32, 512)
# out = transformer_encoder(src)
        
        self.concat_linear = nn.Sequential(
            # nn.Dropout(0.2, inplace=False),
            # nn.LayerNorm(768),
#             nn.LazyLinear(1),
#             nn.ReLU(),
            
#             # nn.LayerNorm(768),
#             nn.Dropout(0.1, inplace=False),
#             nn.LazyLinear(4),
#             nn.ReLU(),
            # nn.LayerNorm(768),
            nn.Linear(768, 512),
            nn.ReLU(),
            
            nn.Linear(512, 256),
            nn.ReLU(),            
            nn.Linear(256, 256),
            nn.ReLU(),
            
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),           
    
            # # nn.Dropout(0.2, inplace=False),
            # nn.Linear(256, 4),
            # nn.ReLU(),
            # nn.Dropout(0.2, inplace=False),
            nn.Linear(16, 1),
            
        )        

    def forward(self, input_ids, attention_mask, token_type_ids, labels):
        
#         # using [CLS]
#         last_hidden_state = self.transformer(
#             input_ids=input_ids, attention_mask=attention_mask, return_dict=True).last_hidden_state[:, 0]
        
        
        # using [Seq]
        X = self.transformer(
            input_ids=input_ids, attention_mask=attention_mask, return_dict=True).last_hidden_state

        # attn_output, attn_output_weights = self.attention_layer(X, X, X)
        
        attn_output = self.encoder(X)
        
        # print(attn_output[:,0,:].shape)
        # return attn_output

# #         # print('attn_output', type(attn_output), attn_output.shape)    
        output = self.concat_linear(attn_output[:, 0, :])
        # print('output.shape', output.shape)
        # output = self.concat_linear(output)

        return output.squeeze(-1)

In [24]:
########### output shape test
test_model = CustomModel(model_name).to(device)
for batch, s in enumerate(train_dataloader):
    print(s['attention_mask'].shape)
    print(s.keys())
    temp_ouput = test_model(**s)
    print(temp_ouput.shape)
    if batch == 0:
        break

Some weights of the model checkpoint at ./../model_saved/checkpoint-140000 were not used when initializing DebertaV2Model: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([40, 128])
dict_keys(['labels', 'input_ids', 'attention_mask', 'token_type_ids'])
torch.Size([40])


#### training

In [33]:
predict_label = 'Elongation_value'

# exp data
train_data, test_data = load_exp_data(pred_prop=predict_label, fes=['text'],
                                      split_ratio=0.7, seed=123)

# text data
data = load_old_data(pred_prop=predict_label)
train_data, test_data  = np.split(data.sample(frac=1, random_state=509, ignore_index=True), [int(0.8*len(data))])
test_data.reset_index(drop=True, inplace=True)
    
# train_data[predict_label] = train_data[predict_label] / 100
# test_data[predict_label] = test_data[predict_label] / 100
    
# train_data[predict_label] = train_data[predict_label]  / train_data[predict_label].abs().max() 
# test_data[predict_label] = test_data[predict_label]  / test_data[predict_label].abs().max()

# train_data[predict_label]= np.log(train_data[predict_label] + 1)
# test_data[predict_label]= np.log(test_data[predict_label] + 1)


print(train_data.shape, test_data.shape)
val_dataloader = DataLoader(CustomDataset(test_data['Text'], test_data.iloc[:, 0],
                                            tokenizer=tokenizer), batch_size=len(test_data))
train_dataloader = DataLoader(CustomDataset(train_data['Text'], train_data.iloc[:, 0],
                                            tokenizer=tokenizer), batch_size=45, shuffle=True, drop_last=True)

(696, 2) (175, 2)


In [35]:
reg_model = CustomModel(model_name).to(device)

######################## training parameters ############################
epoch = 400
lr = 0.01
loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(reg_model.parameters(), weight_decay=0.01) # best 3e-4
# optimizer = torch.optim.LBFGS(params=reg_model.parameters(), lr=lr)

# optimizer = torch.optim.AdamW(reg_model.parameters(), lr=1e-54)
# scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, total_iters=50)
# scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=lr,                                              
                                               # end_factor=1e-4, total_iters=0.8*epoch)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

######################## start training ############################
dt_string = datetime.now().strftime("%d_%m_%Y_%H.%M.%S")
writer = SummaryWriter('runs', flush_secs=20)
test_r2_li = []
for epoch_num in tqdm(range(epoch)):
    # print(f"\n Epoch {epoch_num+1}\n----------------------------------")
    reg_model.train()
    for batch, inputs in enumerate(train_dataloader):
        y = inputs['labels'].unsqueeze(1)
        size = len(y)
    
        preds = reg_model(**inputs)
        train_loss = loss_fn(preds, y)

        result = eval_model(y, preds)
        train_r2 = result['r2']

        # Backpropagation
        optimizer.zero_grad()
        train_loss.backward()        
        optimizer.step()

        
        # if batch % 4 == 0:
        #     train_loss, current = train_loss.item(), (batch + 1) * len(y)
        #     print(f"Train batch loss: {train_loss:>7f}  [{current:>5d}/{size:>5d}]")

    scheduler.step()
    # print(f"Train avg r2: {train_r2}")
    # print(f"train_loss {train_loss}")  

    reg_model.eval()
    num_batches = len(val_dataloader)
    val_loss, val_r2 = 0.0, 0.0
    with torch.no_grad():
        for batch, inputs in enumerate(val_dataloader):
        
            y = inputs['labels'].unsqueeze(1)
            preds = reg_model(**inputs)
            val_loss = loss_fn(preds, y).item()
            result = eval_model(y, preds)
            val_r2 = result['r2']

    val_loss = val_loss/num_batches
    val_r2 = val_r2/num_batches
    # print(val_loss, val_r2)

#     reg_model.eval()
#     with torch.no_grad():
#         for batch, inputs in enumerate(test_dataloader):
#             y = inputs['labels'].unsqueeze(1)
#             preds = reg_model(**inputs)
#             test_r2 = eval_model(y, preds)['r2']
#             print(f"Test r2: {test_r2:>6f}\n")
#             print('*'*10)
#             print(compute_metrics_for_regression((preds, y)), test_r2)
#             print('*'*10)

    writer.add_scalars('model_'+dt_string+'/Loss',
                    tag_scalar_dict = {'train_loss':train_loss,
                                        'val_loss':val_loss},
                    global_step = epoch_num+1)
    writer.add_scalars('model_'+dt_string+'/R2',
                        tag_scalar_dict = {'train_r2':train_r2,
                                           'val_r2':val_r2},
                                           # 'test_r2':test_r2},
                        global_step = epoch_num+1)


    # test_r2_li.append(test_r2)
writer.close()

# return test_r2_li

Some weights of the model checkpoint at ./../model_saved/checkpoint-140000 were not used when initializing DebertaV2Model: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 57%|████████████████████████████▍                     | 228/400 [13:55<10:30,  3.66s/it]

KeyboardInterrupt



#### Check

In [18]:
preds

tensor([653.0493, 653.0498, 653.0502, 653.0490, 653.0504, 653.0505, 653.0501,
        653.0497, 653.0498, 653.0499, 653.0497, 653.0482, 653.0505, 653.0499,
        653.0492, 653.0503], device='cuda:0')

In [19]:
y

tensor([[ 896.0000],
        [ 710.0000],
        [1074.0000],
        [ 596.0000],
        [ 763.0000],
        [ 440.0000],
        [ 945.0000],
        [ 670.0000],
        [ 524.0000],
        [ 337.5000],
        [1389.0000],
        [ 709.0000],
        [ 688.0000],
        [1090.0000],
        [1600.0000],
        [ 387.0000]], device='cuda:0')

In [162]:
reg_model.eval()
for i in range(10):
    with torch.no_grad():
        preds = reg_model(input_ids=inputs['input_ids'][i].unsqueeze(0), 
                          attention_mask=inputs['attention_mask'][i].unsqueeze(0),
                          token_type_ids=inputs['token_type_ids'][i].unsqueeze(0),
                          labels=10)
        print(preds)

tensor([3.8279], device='cuda:0')
tensor([3.8273], device='cuda:0')
tensor([3.8288], device='cuda:0')
tensor([3.8293], device='cuda:0')
tensor([3.8264], device='cuda:0')
tensor([3.8308], device='cuda:0')
tensor([3.8272], device='cuda:0')
tensor([3.8272], device='cuda:0')
tensor([3.8259], device='cuda:0')
tensor([3.8278], device='cuda:0')


In [163]:
reg_model.eval()
with torch.no_grad():
    # for batch, inputs in enumerate(val_dataloader):
    for batch, inputs in enumerate(train_dataloader):        
        y = inputs['labels']
        preds = reg_model(**inputs)
preds

tensor([3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283,
        3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283,
        3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283,
        3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283,
        3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283, 3.8283],
       device='cuda:0')

In [139]:
inputs['input_ids'][0].unsqueeze(0)

tensor([[    1,    14,   205,    52,    23,  3160,    12,   561,  2168,   631,
           400,    24,    75,     9,     8,     6,    71,   542, 21404,    15,
           701,     5,  2617,   307,  1252,    27,  1316,    30,     6,  3877,
           542,  6882,   170,     9,     8,     6,    71,     5, 18142,    24,
          6133,   841,    17,   339,     5,   203,    15,    14,     5, 18142,
           468,    23,   971,  2287,    11,     5,    10,   204,   359,     7,
             5, 25590,    15,    14,     6,   971,  2287,   468,    23,   287,
            19,  2081,    24,    45,     7, 14864,   841,    17,   852,  1287,
             9,   813,    19,   323,   297,    11,   398,   591,     2,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [131]:
inputs

{'labels': tensor([7.0440, 6.9137, 6.9603, 6.8287, 7.1583, 6.8944, 6.9088, 7.1476, 5.7640,
         7.0166, 5.9057, 6.7293, 6.8027, 6.8814, 6.7358, 7.0787, 7.0558, 6.8255,
         7.0731, 6.9912], device='cuda:0'),
 'input_ids': tensor([[  1,  14, 205,  ...,   0,   0,   0],
         [  1,  14, 205,  ...,   0,   0,   0],
         [  1,  14, 205,  ...,   0,   0,   0],
         ...,
         [  1,  14, 205,  ...,   0,   0,   0],
         [  1,  14, 205,  ...,   0,   0,   0],
         [  1,  14, 205,  ...,   0,   0,   0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ...

In [124]:
inputs['input_ids'][0] == inputs['input_ids'][1]

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],

In [125]:
tokenizer.decode(inputs['input_ids'][0])

'[CLS] The experimental steel was melted in vacuum induction melting furnace at first, and then hot-forged. After Some small plates were cut from the as-received hot forged plate, and then solution-treated at 1050 °C for 4 h. The solution-treated specimen was cold rolled to a thickness reduction of 79%. The the cold rolled specimen was performed by tempering at temperature of 590 °C for 15 min, followed by air cooling to room temperature.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [126]:
tokenizer.decode(inputs['input_ids'][1])

'[CLS] The experimental steel was melted in vacuum induction melting furnace at first, and then hot-forged. After Some small plates were cut from the as-received hot forged plate, and then solution-treated at 1050 °C for 4 h. The solution-treated specimen was cold rolled to a thickness reduction of 57%. The the cold rolled specimen was performed by tempering at temperature of 590 °C for 45 min, followed by air cooling to room temperature.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [84]:
temp_inputs_1 = tokenizer.encode_plus(
            test_data.iloc[0,1],
            add_special_tokens=True,
            max_length=128,
            return_token_type_ids=True,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
temp_model_1 = AutoModel.from_pretrained(model_name)
outputs_1 = temp_model(**temp_inputs_1)
outputs_1

Some weights of the model checkpoint at ./../model_saved/checkpoint-140000 were not used when initializing DebertaV2Model: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutput(last_hidden_state=tensor([[[-0.8571, -0.1978, -0.3608,  ..., -0.0811,  0.6296,  0.6619],
         [ 0.1427, -0.2132, -0.9985,  ..., -0.0360, -0.9109, -0.5654],
         [ 0.0971, -0.8286,  0.6010,  ...,  1.8184,  1.3930,  1.0558],
         ...,
         [ 1.2143,  0.3009, -0.3070,  ..., -0.3721,  0.2009, -0.6853],
         [ 1.2143,  0.3009, -0.3070,  ..., -0.3721,  0.2009, -0.6853],
         [ 1.2143,  0.3009, -0.3070,  ..., -0.3721,  0.2009, -0.6853]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [85]:
temp_inputs_2 = tokenizer.encode_plus(
            test_data.iloc[1, 1],
            add_special_tokens=True,
            max_length=128,
            return_token_type_ids=True,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
temp_model_2 = AutoModel.from_pretrained(model_name)
outputs_2 = temp_model(**temp_inputs_2)
outputs_2

Some weights of the model checkpoint at ./../model_saved/checkpoint-140000 were not used when initializing DebertaV2Model: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutput(last_hidden_state=tensor([[[-0.8569, -0.1894, -0.3846,  ..., -0.0709,  0.6550,  0.6459],
         [ 0.1206, -0.1955, -1.0034,  ..., -0.0077, -0.8635, -0.5864],
         [ 0.0852, -0.8316,  0.5898,  ...,  1.8501,  1.3794,  1.0458],
         ...,
         [ 1.2143,  0.3009, -0.3070,  ..., -0.3721,  0.2009, -0.6853],
         [ 1.2143,  0.3009, -0.3070,  ..., -0.3721,  0.2009, -0.6853],
         [ 1.2143,  0.3009, -0.3070,  ..., -0.3721,  0.2009, -0.6853]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [86]:
outputs_1.last_hidden_state.equal(outputs_2.last_hidden_state)

False

In [91]:
reg_model(**temp_inputs_2.to(device), labels=100)

tensor([7.1118], device='cuda:0', grad_fn=<SqueezeBackward1>)

In [109]:
test_data.iloc[0, 1]

'The experimental steel was melted in vacuum induction melting furnace at first, and then hot-forged. After Some small plates were cut from the as-received hot forged plate, and then solution-treated at 1050 °C for 4 h. The solution-treated specimen was cold rolled to a thickness reduction of 79%. The the cold rolled specimen was performed by tempering at temperature of 590 °C for 15 min, followed by air cooling to room temperature.'

In [26]:
result = []
for i in range(20):
    temp_inputs = tokenizer.encode_plus(
                test_data.iloc[i, 1],
                add_special_tokens=True,
                max_length=128,
                return_token_type_ids=True,
                truncation=True,
                padding='max_length',
                return_attention_mask=True,
                return_tensors='pt')
    result += reg_model(**temp_inputs.to(device), labels=100).detach().cpu().numpy().tolist()
eval_model(y.detach().cpu().numpy(), np.array(result))

array([1.54180908, 1.54061759, 1.53958488, 1.53371763, 1.54095399,
       1.54053545, 1.54204571, 1.54078841, 1.54007661, 1.539958  ,
       1.54051137, 1.54105055, 1.53670955, 1.53353095, 1.54121029,
       1.54050779, 1.53656387, 1.5398519 , 1.5402981 , 1.54117703])

In [27]:
y.detach().cpu().numpy()

array([[ 6.88  ],
       [11.2624],
       [15.4273],
       [11.8544],
       [ 8.85  ],
       [ 8.9926],
       [ 7.9   ],
       [11.4785],
       [ 3.6614],
       [12.4898],
       [ 5.32  ],
       [ 4.9911],
       [10.2694],
       [14.8314],
       [ 7.21  ],
       [ 2.7722],
       [12.1586],
       [12.8377],
       [10.0152],
       [ 5.7666]], dtype=float32)

In [28]:
eval_model(y.detach().cpu().numpy(), np.array(result))

{'r2': -4.76816, 'rmse': 8.47942, 'mae': 7.70886}

### pytorch simple

#### dataclass

In [15]:
# containging text and ele features for pytorch
class CustomSimpleDataset(Dataset):
    def __init__(self, embeddings, targets):
        self.embeddings = embeddings.values if isinstance(embeddings, pd.DataFrame) else embeddings
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        embedding = self.embeddings[idx]
        target = self.targets[idx]

        return {
            "labels": torch.tensor(target, dtype=torch.float32).to(device),
            "embeddings": torch.tensor(embedding, dtype=torch.float32).squeeze(0).to(device),
        }
    
###################### CustomSimpleDataset class test
# train_dataloader = DataLoader(CustomDataset(train_data['Text'], train_data.iloc[:, 0],
#                                             tokenizer=tokenizer), batch_size=2)
train_data, test_data = load_exp_data(pred_prop='Elongation_value', fes=['embeddings'],
                                      split_ratio=0.7, seed=666)
print(train_data.shape, test_data.shape)

val_dataloader = DataLoader(CustomSimpleDataset(test_data.iloc[:, 1:], test_data.iloc[:, 0]),
                                    batch_size=2)
train_dataloader = DataLoader(CustomSimpleDataset(train_data.iloc[:, 1:], train_data.iloc[:, 0]),
                                    batch_size=2)


(45, 769) (20, 769)


In [16]:
for batch, s in enumerate(val_dataloader):
    print(s['embeddings'].shape)
    if batch == 0:
        break

torch.Size([2, 768])


#### model

In [39]:
class CustomSimpleModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.concat_linear = nn.Sequential(
            
            nn.Linear(768, 512),
            nn.ReLU(),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            
            # nn.Linear(256, 256),
            # nn.ReLU(),
            
        
#             nn.Linear(256, 256),
#             nn.ReLU(),
#             nn.Linear(256, 256),
#             nn.ReLU(),
            
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),

            nn.Linear(16, 1),            
        )        

    def forward(self, embeddings, labels):
        output = self.concat_linear(embeddings)
        
        return output

In [40]:
########### output shape test
test_model = CustomSimpleModel(model_name).to(device)
for batch, s in enumerate(train_dataloader):
    print(s['embeddings'].shape)
    print(s.keys())
    temp_ouput = test_model(**s)
    print(temp_ouput.shape)
    if batch == 0:
        break

torch.Size([2, 768])
dict_keys(['labels', 'embeddings'])
torch.Size([2, 1])


In [38]:
s

{'labels': tensor([41.7300, 23.5800], device='cuda:0'),
 'embeddings': tensor([[-0.9160, -0.1278, -0.4125,  ...,  0.0113,  0.6152,  0.5559],
         [-0.7693, -0.1105, -0.4356,  ...,  0.0844,  0.5951,  0.5560]],
        device='cuda:0')}

In [41]:
temp_ouput

tensor([[0.2731],
        [0.2729]], device='cuda:0', grad_fn=<AddmmBackward0>)

#### training

In [88]:
predict_label = 'Yield_value'

# exp data
train_data, test_data = load_exp_data(pred_prop=predict_label, fes=['embeddings'],
                                      split_ratio=0.7, seed=123)

# # text data
# data = load_old_data(pred_prop=predict_label)
# train_data, test_data  = np.split(data.sample(frac=1, random_state=509, ignore_index=True), [int(0.8*len(data))])
# test_data.reset_index(drop=True, inplace=True)
    
# train_data[predict_label] = train_data[predict_label] / 100
# test_data[predict_label] = test_data[predict_label] / 100
    
train_data[predict_label] = train_data[predict_label]  / train_data[predict_label].abs().max() 
test_data[predict_label] = test_data[predict_label]  / test_data[predict_label].abs().max()

# train_data[predict_label]= np.log(train_data[predict_label] + 1)
# test_data[predict_label]= np.log(test_data[predict_label] + 1)


print(train_data.shape, test_data.shape)
val_dataloader = DataLoader(CustomSimpleDataset(test_data.iloc[:, 1:], test_data.iloc[:, 0]),
                                    batch_size=len(test_data))
train_dataloader = DataLoader(CustomSimpleDataset(train_data.iloc[:, 1:], train_data.iloc[:, 0]),
                                    batch_size=45, shuffle=True, drop_last=True)

(45, 769) (20, 769)


In [89]:
for i in range(10):

    reg_model = CustomSimpleModel(model_name).to(device)

    ######################## training parameters ############################
    epoch = 1000
    lr = 0.1
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.AdamW(reg_model.parameters(), weight_decay=0.01) # best 3e-4
    # optimizer = torch.optim.LBFGS(params=reg_model.parameters(), lr=lr)

    # optimizer = torch.optim.AdamW(reg_model.parameters(), lr=1e-54)
    # scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, total_iters=50)
    # scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=lr,                                              
    #                                                end_factor=1e-4, total_iters=0.8*epoch)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.8)

    ######################## start training ############################
    dt_string = datetime.now().strftime("%d_%m_%Y_%H.%M.%S")
    writer = SummaryWriter('runs', flush_secs=20)
    test_r2_li = []
    for epoch_num in tqdm(range(epoch)):
        # print(f"\n Epoch {epoch_num+1}\n----------------------------------")
        reg_model.train()
        for batch, inputs in enumerate(train_dataloader):
            y = inputs['labels'].unsqueeze(1)
            size = len(y)

            preds = reg_model(**inputs)
            train_loss = loss_fn(preds, y)

            result = eval_model(y, preds)
            train_r2 = result['r2']

            # Backpropagation
            optimizer.zero_grad()
            train_loss.backward()        
            optimizer.step()


            # if batch % 4 == 0:
            #     train_loss, current = train_loss.item(), (batch + 1) * len(y)
            #     print(f"Train batch loss: {train_loss:>7f}  [{current:>5d}/{size:>5d}]")

        scheduler.step()
        # print(f"Train avg r2: {train_r2}")
        # print(f"train_loss {train_loss}")  

        reg_model.eval()
        num_batches = len(val_dataloader)
        val_loss, val_r2 = 0.0, 0.0
        with torch.no_grad():
            for batch, inputs in enumerate(val_dataloader):

                y = inputs['labels'].unsqueeze(1)
                preds = reg_model(**inputs)
                val_loss = loss_fn(preds, y).item()
                result = eval_model(y, preds)
                val_r2 = result['r2']

        val_loss = val_loss/num_batches
        val_r2 = val_r2/num_batches
        # print(val_loss, val_r2)

    #     reg_model.eval()
    #     with torch.no_grad():
    #         for batch, inputs in enumerate(test_dataloader):
    #             y = inputs['labels'].unsqueeze(1)
    #             preds = reg_model(**inputs)
    #             test_r2 = eval_model(y, preds)['r2']
    #             print(f"Test r2: {test_r2:>6f}\n")
    #             print('*'*10)
    #             print(compute_metrics_for_regression((preds, y)), test_r2)
    #             print('*'*10)

        writer.add_scalars('model_'+dt_string+'/Loss',
                        tag_scalar_dict = {'train_loss':train_loss,
                                            'val_loss':val_loss},
                        global_step = epoch_num+1)
        writer.add_scalars('model_'+dt_string+'/R2',
                            tag_scalar_dict = {'train_r2':train_r2,
                                               'val_r2':val_r2},
                                               # 'test_r2':test_r2},
                            global_step = epoch_num+1)


        # test_r2_li.append(test_r2)
    writer.close()

    # return test_r2_li

100%|████████████████████████████████████████████████| 1000/1000 [00:18<00:00, 54.68it/s]


### hf

#### utils

In [9]:
def compute_metrics_for_regression(eval_preds):
    
    logits, labels = eval_preds
    labels = labels.reshape(-1, 1)
    print(eval_preds)

    mse = mean_squared_error(labels, logits)
    rmse = mean_squared_error(labels, logits, squared=False)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)

    return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "smape": smape}

def preprocess_function(examples):
    norm = BertNormalizer(clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=False)
    examples['text'] = list(map(norm.normalize_str, examples['text']))
    result = tokenizer(examples['text'], padding='max_length', 
                        max_length=512, truncation=True, return_tensors="pt")
    result['label'] = [l for l in examples['label']]
    return result

#### hf DataClass

In [10]:
propery = 'Tensile_value'

train_data, test_data = load_exp_data(pred_prop=propery, fes=['text'],
                                      split_ratio=0.7, seed=789)
df = pd.concat([train_data, test_data])
df.reset_index(drop=True, inplace=True)
df.shape

(65, 2)

In [10]:
propery = 'Tensile_value'

# train_data, test_data = load_exp_data(pred_prop=propery, fes=['text'],
#                                       split_ratio=0.7, seed=789)
# df = pd.concat([train_data, test_data])
# df.reset_index(drop=True, inplace=True)

df = load_old_data(pred_prop=propery, split_ratio=0.75, seed=666)

df.rename(columns={propery:'label', 'Text':'text'}, inplace=True)

dataset_df = Dataset.from_pandas(df).shuffle(seed=456)

tokenized_datasets = dataset_df.map(preprocess_function, batched=True)
dataset = tokenized_datasets.train_test_split(test_size=0.3, seed=666)
train_dataset, val_dataset = dataset['train'], dataset['test']

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
SEED = 666

hf_model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                            num_labels=1).to(device)


Some weights of the model checkpoint at ./../model_saved/checkpoint-140000 were not used when initializing DebertaV2ForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaV2ForSequenceClassification were not initialized from the model chec

In [12]:
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    evaluation_strategy='steps',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    logging_dir = './logs',
    load_best_model_at_end=True,

    learning_rate=2e-2,
    weight_decay=1e-2,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=2e-5,
    max_grad_norm=1.0,
    num_train_epochs=1,
    lr_scheduler_type='linear',
    warmup_ratio=0.1,

    eval_steps=4,
    save_strategy='steps',
    logging_strategy='steps',
    logging_steps=4,
    save_steps=8,
    save_total_limit=1,
    seed=SEED,
    data_seed=SEED,
    fp16=False,
    optim='adamw_torch',
    # max_steps=int(len(train_dataset)*50/64),
    max_steps=600,
    report_to='none'
)

optimizer_kwargs = {
    'params': [p for n, p in model.named_parameters() if not 'deberta' in n],
    'betas': (0.9, 0.99),
    'eps': 2e-5,
    'lr': 1e-4,
    'weight_decay': 1e-4
}
optimizer = AdamW(**optimizer_kwargs)

# Call the Trainer
trainer = Trainer(
    model = hf_model,                         
    args = training_args,                  
    train_dataset = train_dataset,         
    eval_dataset = val_dataset,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics_for_regression,
    # optimizers=(optimizer, None)
)

# Train the model
trainer.train()

# Call the summary
trainer.evaluate()

max_steps is given, it will override any value given in num_train_epochs
The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 609
  Num Epochs = 67
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 600
  Number of trainable parameters = 184422913
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Mse,Rmse,Mae,R2,Smape
4,948795.625000,1123220.750000,1123220.500000,1059.820923,982.527649,-6.115284,51336.610687
8,1062636.125000,1060211.000000,1060211.000000,1029.665527,949.921509,-5.716137,47566.774809
12,1016716.937500,880272.125000,880272.125000,938.228210,849.948242,-4.576275,37719.282443
16,692432.312500,518970.875000,518970.875000,720.396362,601.364746,-2.287534,20478.206107
20,318337.812500,163833.312500,163833.296875,404.763275,304.144318,-0.037838,8105.380725
24,154751.375000,1143660.250000,1143660.125000,1069.420410,992.874512,-6.244763,52400.000000
28,1308715.500000,1123005.125000,1123005.125000,1059.719360,982.417969,-6.113920,51323.358779
32,943839.750000,949395.437500,949395.437500,974.369263,889.682739,-5.014152,41361.309160
36,694494.312500,504650.312500,504650.312500,710.387451,589.512512,-2.196817,19833.538168
40,484001.937500,165328.859375,165328.843750,406.606506,331.439178,-0.047311,8743.654580


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 262
  Batch size = 16


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 262
  Batch size = 16


Saving model checkpoint to ./results\checkpoint-8
Configuration saved in ./results\checkpoint-8\config.json
Model weights saved in ./results\checkpoint-8\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-8\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-8\special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 262
  Batch size = 16


...

Saving model checkpoint to ./results\checkpoint-256
Configuration saved in ./results\checkpoint-256\config.json
Model weights saved in ./results\checkpoint-256\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-256\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-256\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-248] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 262
  Batch size = 16


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 262
  Batch size = 16


Saving model checkpoint to ./results\checkpoint-264
Configuration saved in ./results\checkpoint-264\config.json
Model weights saved in ./results\checkpoint-264\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-264\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-264\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-256] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 262
  Batch size = 16


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 262
  Batch size = 16


Saving model checkpoint to ./results\checkpoint-272
Configuration saved in ./results\checkpoint-272\config.json
Model weights saved in ./results\checkpoint-272\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-272\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-272\special_tokens_map.json

KeyboardInterrupt



In [20]:
y_train, y_val = train_data['Elongation_value'], test_data['Elongation_value']
train_encodings = tokenizer(train_data['Text'].tolist(),
                            add_special_tokens=True,
                            max_length=512,
                            return_token_type_ids=True,
                            truncation=True,
                            padding='max_length',
                            return_attention_mask=True,
                            return_tensors='pt')
valid_encodings = tokenizer(test_data['Text'].tolist(),
                            add_special_tokens=True,
                            max_length=512,
                            return_token_type_ids=True,
                            truncation=True,
                            padding='max_length',
                            return_attention_mask=True,
                            return_tensors='pt')

In [21]:
class MakeTorchData(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        item["labels"] = float(item["labels"])
        return item

    def __len__(self):
        return len(self.labels)
    
# convert our tokenized data into a torch Dataset
train_dataset = MakeTorchData(train_encodings, y_train.ravel())
valid_dataset = MakeTorchData(valid_encodings, y_val.ravel())

#### hf model

In [12]:
class CustomHFModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.name = model
        self.transformer= AutoModel.from_pretrained(self.name)
        self.concat_linear = nn.Sequential(
            nn.Linear(768, 1),
            nn.ReLU(),
        )

    def forward(self, input_ids, attention_mask, token_type_ids, labels):
        
        # using [Seq]
        output = self.transformer(
            input_ids=input_ids, attention_mask=attention_mask, return_dict=True).last_hidden_state[:, 0]
        output = self.concat_linear(output)   
        
#         loss_fct = nn.MSELoss()
#         loss = loss_fct(output.view(-1, 1), labels.view(-1))
        # output = self.concat_linear(output)
        return output.squeeze(-1)

In [41]:
for k,v in train_dataset[0].items():
    print(v.shape)

torch.Size([512])
torch.Size([512])
torch.Size([512])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\shaohan.tian\AppData\Local\Temp\ipykernel_24536\2473836382.py:2 in <module>             │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\shaohan.tian\\AppData\\Local\\Temp\\ipykernel_24536\\2473836382.py'                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'float' object has no attribute 'shape'

In [33]:
text = 'hello'
temp_model = AutoModel.from_pretrained(model_name)
temp_tokenizer = AutoTokenizer.from_pretrained(model_name)

encoded_input = tokenizer(text, padding=True, truncation=False, return_tensors="pt")
# model_output = model(**encoded_input)

test_model(**encoded_input, labels=0)

Some weights of the model checkpoint at ./../model_saved/checkpoint-140000 were not used when initializing DebertaV2Model: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([0.], grad_fn=<SqueezeBackward1>)

In [36]:
encoded_input['input_ids'].shape

torch.Size([1, 5])

In [40]:
input_ids.shape

torch.Size([512])

In [43]:
input_ids = train_dataset[0]['input_ids'].unsqueeze(0)
input_ids.shape

torch.Size([1, 512])

In [45]:
input_ids = train_dataset[0]['input_ids'].unsqueeze(0)
attention_mask = train_dataset[0]['attention_mask'].unsqueeze(0)
token_type_ids = train_dataset[0]['token_type_ids'].unsqueeze(0)
labels = train_dataset[0]['labels']

test_model = CustomHFModel(model_name)
test_model.eval()
test_model(input_ids, attention_mask, token_type_ids, labels)

Some weights of the model checkpoint at ./../model_saved/checkpoint-140000 were not used when initializing DebertaV2Model: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([0.], grad_fn=<SqueezeBackward1>)

#### hf Training

In [12]:
# hf_model = CustomHFModel(model_name).to(device)
hf_model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
                                                   
                                                   
# Specifiy the arguments for the trainer  
training_args = TrainingArguments(
    output_dir ='./results',          
    num_train_epochs = 5,     
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,               
    learning_rate = 2e-3,
    logging_dir = './logs',            
    save_total_limit = 1,
    load_best_model_at_end = True,     
    metric_for_best_model = 'r2',    
    evaluation_strategy = "steps",
    save_strategy = "steps",
    max_steps=2
    
) 

# Call the Trainer
trainer = Trainer(
    model = hf_model,                         
    args = training_args,                  
    train_dataset = train_dataset,         
    eval_dataset = valid_dataset,          
    compute_metrics = compute_metrics_for_regression,     
)

# # Train the model
trainer.train()

# # Call the summary
# trainer.evaluate()

Some weights of the model checkpoint at ./../model_saved/checkpoint-140000 were not used when initializing DebertaV2ForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaV2ForSequenceClassification were not initialized from the model chec

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\shaohan.tian\AppData\Local\Temp\ipykernel_36876\3080396221.py:33 in <module>            │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\shaohan.tian\\AppData\\Local\\Temp\\ipykernel_36876\\3080396221.py'                  │
│                                                                                                  │
│ C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\automl\lib\site-packages\transformers\t │
│ rainer.py:1543 in train                                                                          │
│                                                                                                  │
│   1540 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1541 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1542 │   │   )                                                                                 │
│ ❱ 1543 │   │   return inner_training_loop(                                                       │
│   1544 │   │   │   args=args,                                                                    │
│   1545 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1546 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\automl\lib\site-packages\transformers\t │
│ rainer.py:1791 in _inner_training_loop                                                           │
│                                                                                                  │
│   1788 │   │   │   │   │   with model.no_sync():                                                 │
│   1789 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1790 │   │   │   │   else:                                                                     │
│ ❱ 1791 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1792 │   │   │   │                                                                             │
│   1793 │   │   │   │   if (                                                                      │
│   1794 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\automl\lib\site-packages\transformers\t │
│ rainer.py:2557 in training_step                                                                  │
│                                                                                                  │
│   2554 │   │   │   # loss gets scaled under gradient_accumulation_steps in deepspeed             │
│   2555 │   │   │   loss = self.deepspeed.backward(loss)                                          │
│   2556 │   │   else:                                                                             │
│ ❱ 2557 │   │   │   loss.backward()                                                               │
│   2558 │   │                                                                                     │
│   2559 │   │   return loss.detach()                                                              │
│   2560                                                                                           │
│                                                                                                  │
│ C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\au

In [29]:
torch.cuda.empty_cache()

In [30]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                           num_labels=1,
                                                           ignore_mismatched_sizes=True)

loading configuration file ./../model_saved/checkpoint-140000\config.json
Model config DebertaV2Config {
  "_name_or_path": "./../model_saved/checkpoint-140000",
  "architectures": [
    "DebertaV2ForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "torch_dtype": "float32",
  "transformers_version": "4.

In [32]:
batch_size = 16

args = TrainingArguments(
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=100,
    report_to="none",
    weight_decay=0.01,
    output_dir='./results',
    # metric_for_best_model='accuracy'
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    args,
    train_dataset = train_dataset,         
    eval_dataset = valid_dataset,          
    compute_metrics = compute_metrics_for_regression,
)

PyTorch: setting up devices


In [33]:
# Train the model
trainer.train()

# Call the summary
trainer.evaluate()

***** Running training *****
  Num examples = 45
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 300
  Number of trainable parameters = 184422913


Epoch,Training Loss,Validation Loss,Mse,Rmse,Mae,R2,Smape
1,No log,235.571777,235.571747,15.348347,11.923091,-1.519010,1850.379883
2,No log,173.332855,173.332825,13.165592,9.472361,-0.853478,1225.323828
3,No log,135.523209,135.523209,11.641443,8.628752,-0.449173,1089.562305
4,No log,109.018143,109.018150,10.441175,8.374501,-0.165750,1065.975000
5,No log,95.036667,95.036667,9.748675,8.366055,-0.016244,1063.997656
6,No log,95.601646,95.601662,9.777610,8.699568,-0.022285,1088.907617
7,No log,109.624001,109.624001,10.470148,9.212896,-0.172228,1121.520508
8,No log,131.328949,131.328995,11.459887,9.638322,-0.404324,1140.426563


***** Running Evaluation *****
  Num examples = 20
  Batch size = 16


Saving model checkpoint to ./results\checkpoint-3
Configuration saved in ./results\checkpoint-3\config.json
Model weights saved in ./results\checkpoint-3\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20
  Batch size = 16


Saving model checkpoint to ./results\checkpoint-6
Configuration saved in ./results\checkpoint-6\config.json
Model weights saved in ./results\checkpoint-6\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20
  Batch size = 16


...

Saving model checkpoint to ./results\checkpoint-15
Configuration saved in ./results\checkpoint-15\config.json
Model weights saved in ./results\checkpoint-15\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20
  Batch size = 16


Saving model checkpoint to ./results\checkpoint-18
Configuration saved in ./results\checkpoint-18\config.json
Model weights saved in ./results\checkpoint-18\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20
  Batch size = 16


Saving model checkpoint to ./results\checkpoint-21
Configuration saved in ./results\checkpoint-21\config.json
Model weights saved in ./results\checkpoint-21\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20
  Batch size = 16


Saving model checkpoint to ./results\checkpoint-24
Configuration saved in ./results\checkpoint-24\config.json
Model weights saved in ./results\checkpoint-24\pytorch_model.bin


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\shaohan.tian\AppData\Local\Temp\ipykernel_10608\3488631611.py:2 in <module>             │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\shaohan.tian\\AppData\\Local\\Temp\\ipykernel_10608\\3488631611.py'                  │
│                                                                                                  │
│ C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\automl\lib\site-packages\transformers\t │
│ rainer.py:1543 in train                                                                          │
│                                                                                                  │
│   1540 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1541 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1542 │   │   )                                                                                 │
│ ❱ 1543 │   │   return inner_training_loop(                                                       │
│   1544 │   │   │   args=args,                                                                    │
│   1545 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1546 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\automl\lib\site-packages\transformers\t │
│ rainer.py:1791 in _inner_training_loop                                                           │
│                                                                                                  │
│   1788 │   │   │   │   │   with model.no_sync():                                                 │
│   1789 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1790 │   │   │   │   else:                                                                     │
│ ❱ 1791 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1792 │   │   │   │                                                                             │
│   1793 │   │   │   │   if (                                                                      │
│   1794 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\automl\lib\site-packages\transformers\t │
│ rainer.py:2557 in training_step                                                                  │
│                                                                                                  │
│   2554 │   │   │   # loss gets scaled under gradient_accumulation_steps in deepspeed             │
│   2555 │   │   │   loss = self.deepspeed.backward(loss)                                          │
│   2556 │   │   else:                                                                             │
│ ❱ 2557 │   │   │   loss.backward()                                                               │
│   2558 │   │                                                                                     │
│   2559 │   │   return loss.detach()                                                              │
│   2560                                                                                           │
│                                                                                                  │
│ C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\au